In [1]:
data = []
with open('relations.csv') as f:
    data = [row[:-1].split(',') for row in f][1:]

In [2]:
def process_r(r):
    r = r.replace('-', '_').replace('/', '_or_').replace(' ', '_')
    char_lst=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    
    res = ""
    for i in r:
        if i == '_':
            res += i
        else:
            if i in char_lst:
                res += i
                
    return res.upper()

In [3]:
res = {}
for i in data:
    r = process_r(i[1])
    if r in res.keys():
        res[r].append((i[0], i[2]))
    else:
        res[r] = [(i[0], i[2])]

In [4]:
import csv

In [5]:
for key, val in res.items():
    with open('./data/' + key +'.csv', 'w', newline='') as f:
        f_writer = csv.writer(f)
        f_writer.writerow(['src', 'dst'])
        
        for row in val:
            f_writer.writerow(row)

In [6]:
with open('visulization.cypher', 'w') as f:
    f.write('LOAD CSV WITH HEADERS FROM "file:///nodes.csv" AS line\n')
    f.write('MERGE (p:person{id:line.id, name:line.name, blood_status:line.blood_status, hair_color:line.hair_color, eye_color:line.eye_color, patronus:line.patronus, wand:line.wand, occupation:line.occupation, loyalty:line.loyalty, house:line.house});\n\n')

    f.write('LOAD CSV WITH HEADERS FROM "file:///enemies.csv" AS line\n')
    f.write('MATCH (from:person{id:line.src}),(to:person{id:line.dst})\n')
    f.write('MERGE (from)-[r:ENEMY]->(to);\n\n')
    
    for key in res.keys():
        f.write('LOAD CSV WITH HEADERS FROM "file:///{}.csv" AS line\n'.format(key))
        f.write('MATCH (from:person{id:line.src}),(to:person{id:line.dst})\n')
        f.write('MERGE (from)-[r:{}]->(to);\n\n'.format(key))